# Linear Regression Toolkit - Experiments

This notebook demonstrates the Linear Regression toolkit with three experiments:
1. Straight Line with Noise
2. Collinearity and Ridge Regularization
3. Polynomial Regression


In [ ]:
import sys
import os
# Add parent directory to path
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), '..'))

import numpy as np
import matplotlib.pyplot as plt
from linear_regression.linear_models import LinearRegressionClosedForm
from linear_regression.metrics import mse, r2_score
from linear_regression.selection import train_test_split
from linear_regression.plotting import plot_predictions, plot_residuals

np.random.seed(42)


## Experiment 1: Straight Line with Noise

Generate synthetic data: $y = 3 + 2x + \epsilon$, where $\epsilon \sim N(0, 1)$.


In [ ]:
# Generate synthetic data: y = 3 + 2*x + noise
n_samples = 100
X = np.linspace(0, 10, n_samples).reshape(-1, 1)
y_true = 3 + 2 * X.ravel()
y = y_true + np.random.normal(0, 1, n_samples)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model with alpha=0.0 (no regularization)
model = LinearRegressionClosedForm(fit_intercept=True, alpha=0.0)
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Report results
print("Learned coefficients:")
print(f"  Intercept: {model.intercept_:.4f} (true: 3.0)")
print(f"  Coefficient: {model.coef_[0]:.4f} (true: 2.0)")
print(f"\nTrain MSE: {mse(y_train, y_train_pred):.4f}")
print(f"Test MSE: {mse(y_test, y_test_pred):.4f}")
print(f"Train R²: {r2_score(y_train, y_train_pred):.4f}")
print(f"Test R²: {r2_score(y_test, y_test_pred):.4f}")


In [ ]:
# Plot predicted vs true values
plot_predictions(y_test, y_test_pred, title="Experiment 1: Predictions vs True Values")
plt.show()


## Experiment 2: Collinearity and Ridge Regularization

Create two highly correlated features and fit models with different alpha values.


In [ ]:
# Create collinear features: x2 = x1 + 0.01 * noise
n_samples = 200
x1 = np.random.randn(n_samples)
x2 = x1 + 0.01 * np.random.randn(n_samples)
X = np.column_stack([x1, x2])

# True relationship: y = x1 + x2 + noise
y = x1 + x2 + 0.1 * np.random.randn(n_samples)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit models with different alpha values
alphas = [0.0, 1e-3, 1e-2, 1e-1, 1.0]
results = []

for alpha in alphas:
    model = LinearRegressionClosedForm(alpha=alpha)
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    test_mse = mse(y_test, y_test_pred)
    coef_norm = np.linalg.norm(model.coef_)
    results.append({
        'alpha': alpha,
        'test_mse': test_mse,
        'coef_norm': coef_norm,
        'coef': model.coef_.copy()
    })
    print(f"Alpha={alpha:6.4f}: Test MSE={test_mse:.4f}, ||coef||₂={coef_norm:.4f}")

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

alphas_plot = [r['alpha'] for r in results]
mse_plot = [r['test_mse'] for r in results]
norm_plot = [r['coef_norm'] for r in results]

ax1.semilogx(alphas_plot, mse_plot, 'o-')
ax1.set_xlabel('Alpha (regularization strength)')
ax1.set_ylabel('Test MSE')
ax1.set_title('Test MSE vs Alpha')
ax1.grid(True, alpha=0.3)

ax2.semilogx(alphas_plot, norm_plot, 'o-')
ax2.set_xlabel('Alpha (regularization strength)')
ax2.set_ylabel('||coef||₂')
ax2.set_title('Coefficient Norm vs Alpha')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Discussion

As alpha increases, the L2 regularization penalty becomes stronger. This causes:
1. **Coefficient shrinkage**: The weight vector norm ||coef||₂ decreases, making the model simpler and less prone to overfitting.
2. **Test MSE behavior**: Initially, regularization may improve generalization by reducing overfitting. However, if alpha becomes too large, the model becomes underfitted and test MSE increases.
3. **Stability**: With collinear features, regularization helps stabilize the solution by preventing coefficients from becoming extremely large.

In this experiment, we can see that moderate regularization (alpha around 0.01-0.1) helps control the coefficient magnitudes while maintaining good predictive performance.


## Experiment 3: Polynomial Regression

Generate nonlinear data: $y = 1 + 2x - 0.3x^2 + \epsilon$. Compare degree 1, 2, and 5 polynomial features.


In [ ]:
# Generate nonlinear data: y = 1 + 2*x - 0.3*x^2 + noise
n_samples = 100
X = np.linspace(-5, 5, n_samples).reshape(-1, 1)
y_true = 1 + 2 * X.ravel() - 0.3 * X.ravel() ** 2
y = y_true + np.random.normal(0, 0.5, n_samples)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Helper function to create polynomial features
def polynomial_features(X, degree):
    """Create polynomial features up to given degree."""
    features = [X ** d for d in range(1, degree + 1)]
    return np.column_stack(features)

# Compare different polynomial degrees
degrees = [1, 2, 5]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, degree in enumerate(degrees):
    # Create polynomial features
    X_train_poly = polynomial_features(X_train, degree)
    X_test_poly = polynomial_features(X_test, degree)
    
    # Fit model
    model = LinearRegressionClosedForm(alpha=0.0)
    model.fit(X_train_poly, y_train)
    
    # Predictions for plotting
    X_plot = np.linspace(-5, 5, 200).reshape(-1, 1)
    X_plot_poly = polynomial_features(X_plot, degree)
    y_plot_pred = model.predict(X_plot_poly)
    
    # Test predictions
    y_test_pred = model.predict(X_test_poly)
    test_mse = mse(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Plot
    ax = axes[idx]
    ax.scatter(X_train, y_train, alpha=0.5, label='Train', s=20)
    ax.scatter(X_test, y_test, alpha=0.5, label='Test', s=20, marker='^')
    ax.plot(X_plot, y_true, 'g--', label='True function', linewidth=2)
    ax.plot(X_plot, y_plot_pred, 'r-', label='Fitted', linewidth=2)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'Degree {degree}\nTest MSE: {test_mse:.4f}, R²: {test_r2:.4f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Comment

**Which degree fits best and why?**

- **Degree 1 (Linear)**: Underfits the data. The true function is quadratic, so a linear model cannot capture the curvature. Test MSE is high.

- **Degree 2 (Quadratic)**: Fits best! This matches the true underlying function (y = 1 + 2x - 0.3x²). The model has just enough flexibility to capture the quadratic relationship without overfitting. Test MSE is lowest and R² is highest.

- **Degree 5 (High-order polynomial)**: Overfits the data. While it can fit the training data very well, it captures noise and generalizes poorly to test data. The high-order terms cause the curve to wiggle unnecessarily, leading to higher test MSE.

**Conclusion**: Degree 2 is optimal because it matches the true model complexity. Higher degrees overfit, while lower degrees underfit.
